# **Project title**: South St. Petersburg Community Redevelopment Area Property Value Analysis


## Team Members(*Group 5)*:
 - Rahul R. Vemparala
 - Sainath Vancharla
 - Ravali Venkatayogi



 ## **Topic**: Analyze taxable property values and trends between 2015 and 2022

In [ ]:
"""
Author: Rahul R.Vemparala
Date: 2024-04-23
Description: Perform EDA analysis on Tax property Data for St.Pitsburg City.
"""


'\nAuthor: Rahul R.Vemparala\nDate: 2024-04-23\nDescription: Perform EDA analysis on Tax property Data for St.Pitsburg City.\n'

# Dataset Citations

**1. 2015 Property Appraiser/Department of Review Data:**

* Source: University of South Florida (USF) Box (https://www.coursehero.com/sitemap/schools/57-University-of-South-Florida/courses/8518206-QMB6358/)

**2. 2022 Property Appraiser/Department of Review Data:**

* Source: University of South Florida (USF) Box (https://usf.account.box.com/)

**3. Property Categorization:**

* Source: University of South Florida (USF) Box (https://www.usf.edu/business-finance/controller/documents/propertymanual.pdf)

**4. Neighborhood Map:**

* Source: University of South Florida (USF) Box (https://www.usf.edu/administrative-services/parking/documents/campus-map-directory.pdf)




In [ ]:
# Dataset Imports
from google.colab import files

def upload_files():
    """
    Allows the user to upload files to Colab's temporary storage.

    Returns:
        dict: A dictionary where the keys are the filenames and the values are the file contents.
    """
    return list(files.upload().keys())

In [ ]:
uploaded = []

while True:
    user_input = input("Say 'yes' to start an Upload or type 'done' to finish: ")

    if user_input.lower() == 'done':
        break

    uploaded.append(upload_files()[0])

Say 'yes' to start an Upload or type 'done' to finish: 


Saving 2015_NewNeighb_Boundaries with coordinates (1).xlsx to 2015_NewNeighb_Boundaries with coordinates (1).xlsx
Say 'yes' to start an Upload or type 'done' to finish: 


Saving 2022_NewNeighbBoundaries with coordinates (2).xlsx to 2022_NewNeighbBoundaries with coordinates (2).xlsx
Say 'yes' to start an Upload or type 'done' to finish: done


In [ ]:
import pandas as pd
import numpy as np


for x in uploaded:
  print(x)


2015_NewNeighb_Boundaries with coordinates (1).xlsx
2022_NewNeighbBoundaries with coordinates (2).xlsx


In [ ]:

df_15 = pd.read_excel(uploaded[0])
df_22 = pd.read_excel(uploaded[1])

# df_cat = pd.read_excel(uploaded[0])

In [ ]:
df_15.info()

In [ ]:
df_22.info()

In [ ]:
df_15.isnull().sum()



In [ ]:
df_22.isnull().sum()


# Handling missing  Data

## *Observations*:
Missing Data needs to addressed prior to performing any analysis, below are the operations done to handle missing data

### *action items*:

1. to address the missing  Latitude and Longitude, we need to first handle missing physical address,
2.  Taxable Value – Non-School District is 0 for these missing Addresses, hence it is safe to drop the rows (88 records)

In [ ]:
# to address the missing  Latitude and Longitude, we need to first handle missing physical address,
# Taxable Value – Non-School District is 0 for these missing Addresses, hence it is safe to drop the rows (88 records)

df_15 = df_15.dropna(subset=['Physical Address'])

In [ ]:
## Addressing Missing Data in 2015 property value data:


# Fill missing longitude and latitude values with neighborhood means
df_15['Longitude'] = df_15.groupby('Neighborhood')['Longitude'].transform(lambda x: x.fillna(x.mean()))
df_15['Latitude'] = df_15.groupby('Neighborhood')['Latitude'].transform(lambda x: x.fillna(x.mean()))


In [ ]:
df_15.isnull().sum()

In [ ]:
df_15 = df_15.dropna(subset=['Property Use', 'PIN'])

In [ ]:
# addressing missing values for 2022 tax property values:

df_22 = df_22.dropna(subset=['Physical Address'])
df_22['Longitude'] = df_22.groupby('Neighborhood')['Longitude'].transform(lambda x: x.fillna(x.mean()))


In [ ]:
df_22.isnull().sum()

In [ ]:
df_22 = df_22.dropna(subset=['Property Use'])

# Descrepencies in "Neighborhood column"

## *Observations*:
1. The values in the Neighborhood columns when compared between 2015 and 2022, have descrepencies which was causing the join operation to fail.
2. Upon inspection we,noticed that there is suffix for Majority of the Neighborhood names have a suffix "NBRHD ASSN" ( Neighborhood association)
3. Comparing other join keys we noticed not much difference in the geographical plane( Latitude and Longitude coordinates)

*Decision:*
4. Hene we considered chopping the suffix for the Neighborhood column in 2022 tax data.

In [ ]:
# Remove "NBRHD ASSN" and strip leading/trailing white spaces from the "Neighborhood" column


df_22['Neighborhood'] = df_22['Neighborhood'].str.replace(' NBRHD ASSN', '').str.strip()

# Deciding Group keys

## *Observations:*

1.  In order to compare results between 2015 and 2022 Property tax data we need to ensure we join them on proper selection of identifiers, as we are comparing properties with same Physical address.
2. Hence based on analysis and research below are group keys we decided to work with in joining both the tax property datasets.
  - Property Use
  - Neighborhood
  - Physical Address
  - Physical Zip Code


# Multiple Records with Property use, Neighborhood, same address, zipcode

## *Observations*:
1. to have the analysis done correctly, comparing the same properties over different times is needed, that means we have to compare a Property with same Neighborhood, address,zip code etc for 2 diff times( 2015 and 2022)
2. to acheive that we implemented group keys to have a proper merge between the datasets,
3. BUt we noticed duplicate values in the dataset, which means same proerty use, Neighborhood, Address, Zipcode has more than one entry ( more than one tax value)

## *actions:*
1. In our case ,we just picked the record with the maximum Tax Value ( NOn school district)  and droped the rest.

In [ ]:

group_keys = ['Property Use',
              'Neighborhood','Physical Address', 'Physical Zip Code',]

# Grouping by identifiers and selecting the row with
# maximum "Taxable Value – Non-School District"

max_taxable_value_index = \
df_15.groupby(group_keys)['Taxable Value – Non-School District'].idxmax()

# Selecting the rows with maximum taxable value per group
df_15_filtered = df_15.loc[max_taxable_value_index]

# Displaying the final filtered DataFrame
df_15_filtered.shape

(13744, 20)

In [ ]:


# Grouping by identifiers and selecting the row with
# maximum "Taxable Value – Non-School District"

max_taxable_value_index = \
df_22.groupby(group_keys)['Taxable Value – Non-School District'].idxmax()

# Selecting the rows with maximum taxable value per group
df_22_filtered = df_22.loc[max_taxable_value_index]

# Displaying the final filtered DataFrame
df_22_filtered.shape

# Exporting datasets:

Exporting the pre-processed datasets, for furthur analysis in Power BI

In [ ]:
df_15_filtered.to_csv('Property tax 2015 pre-processed.csv', index=False)
df_22_filtered.to_csv('Property tax 2022 pre-processed.csv', index=False)

In [ ]:
df_22_filtered.info()

# Merge datasets (Optional)  for cross check:

## Data validation section: ( optional)


In [ ]:
import pandas as pd

# Perform the inner join and preserve the column names
merged_df = pd.merge(df_22_filtered, df_15_filtered,
                     left_on=['Property Use', 'Neighborhood',
                              'Physical Address', 'Physical Zip Code'],
                     right_on=['Property Use', 'Neighborhood',
                               'Physical Address', 'Physical Zip Code'],
                     how='left',
                     suffixes=('_22', '_15'))

# Reorder the columns in alphabetical order
merged_df = merged_df[sorted(merged_df.columns)]


In [ ]:
# Rename the join key columns in both DataFrames with suffixes
df_22_filtered_renamed = df_22_filtered.rename(columns=lambda x: x + '_22')
df_15_filtered_renamed = df_15_filtered.rename(columns=lambda x: x + '_15')

# Perform the inner join with suffixes applied to all columns
merged_df = pd.merge(df_22_filtered_renamed, df_15_filtered_renamed,
                     left_on=['Property Use_22', 'Neighborhood_22',
                              'Physical Address_22', 'Physical Zip Code_22'],
                     right_on=['Property Use_15', 'Neighborhood_15',
                               'Physical Address_15', 'Physical Zip Code_15'],
                     how='outer',
                     suffixes=('_22', '_15'))

# Reorder the columns in alphabetical order
merged_df = merged_df.reindex(sorted(merged_df.columns), axis=1)


In [ ]:
#q1:
import pandas as pd

# Assuming merged_df is your merged DataFrame containing data for both 2015 and 2022

# Group by 'Neighborhood' and calculate total taxable property values for each year
total_taxable_values_2015 = merged_df.groupby('Neighborhood_15')['Taxable Value – Non-School District_15'].sum()
total_taxable_values_2022 = merged_df.groupby('Neighborhood_22')['Taxable Value – Non-School District_22'].sum()

# Combine the results into a single DataFrame for comparison
total_taxable_values = pd.concat([total_taxable_values_2015, total_taxable_values_2022], axis=1)
total_taxable_values.columns = ['Total Taxable Value 2015', 'Total Taxable Value 2022']

# Display the results
total_taxable_values


,Total Taxable Value 2015,Total Taxable Value 2022
BARTLETT PARK,15240874,54038891
CAMPBELL PARK,14360342,34869417
CASLER HEIGHTS,2087395,7970282
CENTRAL OAK PARK,36873374,107238471
CHILDS PARK,47410803,149801262
CLAM BAYOU,402522,922411
CROMWELL HEIGHTS,4640507,16620712
FRUITLAND HEIGHTS,2457919,8327161
HARBORDALE,10981305,36795048
HIGHLAND OAKS,13380988,46374454


In [ ]:
# Calculate total taxable property values for each year in each neighborhood
total_taxable_values_2015 = df_15_filtered.groupby('Neighborhood')['Taxable Value – Non-School District'].sum()
total_taxable_values_2022 = df_22_filtered.groupby('Neighborhood')['Taxable Value – Non-School District'].sum()

# Combine the results into a single DataFrame for comparison
taxable_values = pd.concat([total_taxable_values_2015, total_taxable_values_2022], axis=1)
taxable_values.columns = ['tax Value 2015', 'tax Value 2022']

# Display the results
taxable_values

,tax Value 2015,tax Value 2022
Neighborhood,,
BARTLETT PARK,21022904,70183340
CAMPBELL PARK,17771796,64483647
CASLER HEIGHTS,3283140,10836010
CENTRAL OAK PARK,74413231,178172008
CHILDS PARK,67435705,192295130
CLAM BAYOU,408817,1129764
CROMWELL HEIGHTS,7465083,21855697
FRUITLAND HEIGHTS,3440924,10356194
HARBORDALE,15420263,47868897


In [ ]:
# Concatenate the two DataFrames side by side
comparison_df = pd.concat([total_taxable_values, taxable_values], axis=1)

# Display the comparison DataFrame
comparison_df


,Total Taxable Value 2015,Total Taxable Value 2022,tax Value 2015,tax Value 2022
Neighborhood,,,,
BARTLETT PARK,21022904,70183340,21022904,70183340
CAMPBELL PARK,17771796,64483647,17771796,64483647
CASLER HEIGHTS,3283140,10836010,3283140,10836010
CENTRAL OAK PARK,74413231,178172008,74413231,178172008
CHILDS PARK,67435705,192295130,67435705,192295130
CLAM BAYOU,408817,1129764,408817,1129764
CROMWELL HEIGHTS,7465083,21855697,7465083,21855697
FRUITLAND HEIGHTS,3440924,10356194,3440924,10356194
HARBORDALE,15420263,47868897,15420263,47868897
